In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1
)

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=80,
    return_messages=True
)

cache_dir=LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}")
])

chain = {"context": retriver, "question" : RunnablePassthrough()} | prompt | llm 

question1 = chain.invoke("Is Aaronson guilty?")
memory.save_context({"question":"Is Aaronson guilty?"},{"output": question1.content})
question2 = chain.invoke("What message did he write in the table?")
memory.save_context({"question":"What message did he write in the table?"},{"output": question2.content})
question3 = chain.invoke("Who is Julia?")
memory.save_context({"question":"Who is Julia?"},{"output": question3.content})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Is Aaronson guilty?'),
  AIMessage(content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'),
  HumanMessage(content='What message did he write in the table?'),
  AIMessage(content='He wrote "2+2=5" in the dust on the table.'),
  HumanMessage(content='Who is Julia?'),
  AIMessage(content='Julia is a character in the text who is significant to the protagonist.')]}